# Firefox Launcher Secondary Tab Debugging

This notebook will help debug issues with opening secondary browser tabs in JupyterLab using the Firefox launcher extension.

## Problem Overview
- Difficulty opening multiple Firefox tabs/sessions
- Need to investigate multi-session support
- Verify proper process isolation and cleanup

Let's systematically check each component to identify the root cause.

In [ ]:
# Import Required Libraries
import os
import sys
import json
import time
import logging
import psutil
import subprocess
import requests
from pathlib import Path
from typing import Dict, Any, List

# Configure logging for better debugging output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All required libraries imported successfully")
print(f"📍 Current working directory: {os.getcwd()}")
print(f"🐍 Python executable: {sys.executable}")
print(f"🏠 Home directory: {Path.home()}")

✅ All required libraries imported successfully
📍 Current working directory: /home/bdx/allcode/github/vantagecompute/jfl
🐍 Python executable: /home/bdx/allcode/github/vantagecompute/jfl/.venv/bin/python
🏠 Home directory: /home/bdx


: 

In [ ]:
# Inspect Current Firefox Handler State
print("🔍 STEP 1: Inspecting Firefox Handler State")
print("="*50)

# Check if we can import the Firefox handler
try:
    from jupyterlab_firefox_launcher.firefox_handler import FirefoxLauncherHandler, _find_free_port
    print("✅ Successfully imported FirefoxLauncherHandler")
    
    # Check current active sessions
    if hasattr(FirefoxLauncherHandler, '_active_sessions'):
        active_sessions = FirefoxLauncherHandler._active_sessions
        print(f"📊 Current active sessions: {len(active_sessions)}")
        
        if active_sessions:
            print("🔍 Active session details:")
            for port, session_info in active_sessions.items():
                print(f"   Port {port}: {session_info}")
        else:
            print("ℹ️  No active sessions found")
    else:
        print("⚠️  _active_sessions attribute not found")
        
    # Check if lock exists
    if hasattr(FirefoxLauncherHandler, '_xpra_startup_lock'):
        lock_status = FirefoxLauncherHandler._xpra_startup_lock
        print(f"🔒 Startup lock status: {lock_status}")
    else:
        print("⚠️  _xpra_startup_lock not found")
        
except ImportError as e:
    print(f"❌ Failed to import Firefox handler: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

print()

In [ ]:
# Test Multi-Session Support via API
print("🧪 STEP 2: Testing Multi-Session Support")
print("="*50)

# Find JupyterLab server URL - common locations
possible_urls = [
    "http://localhost:8888",
    "http://localhost:8889", 
    "http://127.0.0.1:8888",
    "http://127.0.0.1:8889"
]

jupyter_base_url = None
for url in possible_urls:
    try:
        # Try to connect without authentication first
        response = requests.get(f"{url}/api/status", timeout=2)
        if response.status_code in [200, 403]:  # 403 means server exists but needs auth
            jupyter_base_url = url
            print(f"✅ Found JupyterLab server at: {jupyter_base_url}")
            break
    except requests.exceptions.RequestException:
        continue

if not jupyter_base_url:
    print("❌ Could not find running JupyterLab server")
    print("💡 Please ensure JupyterLab is running and try again")
    print("   Example: uv run jupyter lab --no-browser --port=8888")
else:
    # Test Firefox launcher API endpoints
    firefox_api_url = f"{jupyter_base_url}/firefox-launcher/api/firefox"
    
    print(f"🔍 Testing Firefox launcher API at: {firefox_api_url}")
    
    try:
        # Test GET request (status check)
        response = requests.get(f"{firefox_api_url}?status=true", timeout=10)
        print(f"📊 GET Status: {response.status_code}")
        if response.status_code == 200:
            data = response.json()
            print(f"   Response: {data}")
        else:
            print(f"   Response text: {response.text[:200]}")
            
    except Exception as e:
        print(f"❌ GET request failed: {e}")
    
    # Note: POST requests require authentication tokens
    print("⚠️  POST requests require proper authentication")
    print("   To test session creation, use the JupyterLab UI or properly authenticated requests")

print()

In [ ]:
# Debug Browser Tab Creation Components
print("🔧 STEP 3: Debugging Browser Tab Creation")
print("="*50)

# Check system dependencies
print("1. System Dependencies Check:")
print("-" * 30)

# Check for Xpra
xpra_path = subprocess.run(['which', 'xpra'], capture_output=True, text=True)
if xpra_path.returncode == 0:
    print(f"✅ Xpra found at: {xpra_path.stdout.strip()}")
    
    # Check Xpra version
    try:
        version_result = subprocess.run(['xpra', '--version'], capture_output=True, text=True, timeout=5)
        if version_result.returncode == 0:
            print(f"   Version: {version_result.stdout.strip()}")
    except Exception as e:
        print(f"   ⚠️  Could not get version: {e}")
else:
    print("❌ Xpra not found - this is required for Firefox launcher")

# Check for Firefox
firefox_path = subprocess.run(['which', 'firefox'], capture_output=True, text=True)
if firefox_path.returncode == 0:
    print(f"✅ Firefox found at: {firefox_path.stdout.strip()}")
else:
    print("❌ Firefox not found - this is required")

# Check for firefox-xstartup script
print("\n2. Firefox Startup Script Check:")
print("-" * 30)

firefox_startup_path = subprocess.run(['which', 'firefox-xstartup'], capture_output=True, text=True)
if firefox_startup_path.returncode == 0:
    print(f"✅ firefox-xstartup found at: {firefox_startup_path.stdout.strip()}")
else:
    print("⚠️  firefox-xstartup not in PATH, checking development location...")
    
    # Check development location
    dev_script = Path(__file__).parent / "scripts" / "firefox-xstartup"
    if dev_script.exists():
        print(f"✅ Found in development location: {dev_script}")
        print(f"   Executable: {dev_script.is_file() and os.access(dev_script, os.X_OK)}")
    else:
        print("❌ firefox-xstartup script not found anywhere")

# Check session directory structure
print("\n3. Session Directory Structure:")
print("-" * 30)

session_base = Path.home() / '.firefox-launcher' / 'sessions'
print(f"📁 Base session directory: {session_base}")
print(f"   Exists: {session_base.exists()}")

if session_base.exists():
    sessions = list(session_base.glob('session-*'))
    print(f"   Active session directories: {len(sessions)}")
    for session_dir in sessions:
        print(f"     {session_dir.name}")

print()

In [ ]:
# Check Server Proxy Configuration
print("🌐 STEP 4: Checking Server Proxy Configuration")
print("="*50)

# Test port allocation
print("1. Port Allocation Test:")
print("-" * 25)

try:
    from jupyterlab_firefox_launcher.firefox_handler import _find_free_port
    
    # Test finding multiple free ports
    ports = []
    for i in range(3):
        port = _find_free_port()
        ports.append(port)
        print(f"   Test {i+1}: Allocated port {port}")
    
    print(f"✅ Port allocation working - got ports: {ports}")
    
    # Check for port conflicts
    unique_ports = len(set(ports))
    if unique_ports == len(ports):
        print("✅ All ports are unique - no conflicts")
    else:
        print(f"⚠️  Port conflicts detected - only {unique_ports} unique ports out of {len(ports)}")
        
except Exception as e:
    print(f"❌ Port allocation test failed: {e}")

# Check jupyter-server-proxy configuration
print("\n2. Jupyter Server Proxy Configuration:")
print("-" * 40)

try:
    # Check if jupyter-server-proxy is available
    import jupyter_server_proxy
    print(f"✅ jupyter-server-proxy version: {jupyter_server_proxy.__version__}")
    
    # Try to import server proxy config
    from jupyterlab_firefox_launcher.firefox_handler import get_server_proxy_config
    config = get_server_proxy_config()
    
    print("📋 Server proxy configuration:")
    for key, value in config.items():
        print(f"   {key}: {type(value).__name__}")
        if isinstance(value, dict):
            for subkey, subvalue in value.items():
                if callable(subvalue):
                    print(f"     {subkey}: {type(subvalue).__name__} (callable)")
                else:
                    print(f"     {subkey}: {subvalue}")
    
except ImportError as e:
    print(f"❌ jupyter-server-proxy import failed: {e}")
except Exception as e:
    print(f"❌ Configuration check failed: {e}")

print()

In [ ]:
# Monitor Process Management
print("⚙️ STEP 5: Process Management Analysis")
print("="*50)

# Check for existing Firefox/Xpra processes
print("1. Current Firefox/Xpra Processes:")
print("-" * 35)

firefox_processes = []
xpra_processes = []

try:
    for proc in psutil.process_iter(['pid', 'name', 'cmdline', 'status']):
        proc_name = proc.info['name'].lower() if proc.info['name'] else ""
        
        if 'firefox' in proc_name:
            firefox_processes.append(proc.info)
        elif 'xpra' in proc_name:
            xpra_processes.append(proc.info)

    print(f"🦊 Firefox processes found: {len(firefox_processes)}")
    for proc in firefox_processes:
        print(f"   PID {proc['pid']}: {proc['name']} (status: {proc['status']})")
        
    print(f"📺 Xpra processes found: {len(xpra_processes)}")
    for proc in xpra_processes:
        print(f"   PID {proc['pid']}: {proc['name']} (status: {proc['status']})")
        
    if not firefox_processes and not xpra_processes:
        print("ℹ️  No active Firefox or Xpra processes found")
        
except Exception as e:
    print(f"❌ Process enumeration failed: {e}")

# Check system resources
print("\n2. System Resources:")
print("-" * 20)

try:
    # Memory usage
    memory = psutil.virtual_memory()
    print(f"💾 Memory: {memory.percent:.1f}% used ({memory.used // 1024**3}GB / {memory.total // 1024**3}GB)")
    
    # CPU usage
    cpu_percent = psutil.cpu_percent(interval=1)
    print(f"🖥️  CPU: {cpu_percent:.1f}% used")
    
    # Load average (if available)
    try:
        load_avg = psutil.getloadavg()
        print(f"📊 Load average: {load_avg[0]:.2f}, {load_avg[1]:.2f}, {load_avg[2]:.2f}")
    except AttributeError:
        print("📊 Load average not available on this platform")
        
except Exception as e:
    print(f"❌ Resource check failed: {e}")

# Check for potential conflicts
print("\n3. Potential Conflicts Check:")
print("-" * 30)

# Check for other remote desktop software
conflict_processes = []
conflict_names = ['vnc', 'rdp', 'teamviewer', 'anydesk', 'chrome-remote', 'nomachine']

try:
    for proc in psutil.process_iter(['pid', 'name']):
        proc_name = proc.info['name'].lower() if proc.info['name'] else ""
        for conflict_name in conflict_names:
            if conflict_name in proc_name:
                conflict_processes.append(proc.info)
                break

    if conflict_processes:
        print("⚠️  Potential conflicting processes found:")
        for proc in conflict_processes:
            print(f"   PID {proc['pid']}: {proc['name']}")
    else:
        print("✅ No obvious conflicting processes detected")
        
except Exception as e:
    print(f"❌ Conflict check failed: {e}")

print()

In [ ]:
# Test Session Cleanup Functionality
print("🧹 STEP 6: Session Cleanup Testing")
print("="*50)

# Test cleanup functions
print("1. Testing Cleanup Functions:")
print("-" * 30)

try:
    from jupyterlab_firefox_launcher.firefox_handler import _cleanup_firefox_profile
    
    # Test cleanup function with a fake port (safe test)
    test_port = 99999  # Very unlikely to be a real session
    
    print(f"🧪 Testing cleanup function with fake port {test_port}")
    result = _cleanup_firefox_profile(test_port)
    print(f"   Cleanup result: {result} (expected: True for non-existent session)")
    
except Exception as e:
    print(f"❌ Cleanup function test failed: {e}")

# Check for orphaned session directories
print("\n2. Orphaned Session Directory Check:")
print("-" * 35)

session_base = Path.home() / '.firefox-launcher' / 'sessions'

if session_base.exists():
    session_dirs = list(session_base.glob('session-*'))
    print(f"📁 Found {len(session_dirs)} session directories")
    
    if session_dirs:
        # Check if corresponding processes exist
        orphaned_sessions = []
        
        for session_dir in session_dirs:
            # Extract port from directory name
            try:
                port_str = session_dir.name.replace('session-', '')
                port = int(port_str)
                
                # Check if any active sessions match this port
                if hasattr(FirefoxLauncherHandler, '_active_sessions'):
                    active_sessions = FirefoxLauncherHandler._active_sessions
                    if port not in active_sessions:
                        orphaned_sessions.append((port, session_dir))
                
            except ValueError:
                print(f"   ⚠️  Invalid session directory name: {session_dir.name}")
        
        if orphaned_sessions:
            print(f"🗑️  Found {len(orphaned_sessions)} potentially orphaned sessions:")
            for port, session_dir in orphaned_sessions:
                print(f"   Port {port}: {session_dir}")
                
                # Check directory contents
                try:
                    contents = list(session_dir.iterdir())
                    print(f"     Contents: {len(contents)} items")
                    for item in contents[:3]:  # Show first 3 items
                        print(f"       - {item.name}")
                    if len(contents) > 3:
                        print(f"       ... and {len(contents) - 3} more")
                except Exception as e:
                    print(f"     ❌ Could not list contents: {e}")
        else:
            print("✅ No orphaned session directories found")
else:
    print("ℹ️  No session base directory found")

# Test manual cleanup (commented out for safety)
print("\n3. Manual Cleanup Options:")
print("-" * 25)
print("🔧 To manually clean up orphaned sessions:")
print("   1. Stop JupyterLab")
print("   2. Run: rm -rf ~/.firefox-launcher/sessions/session-*")
print("   3. Restart JupyterLab")
print("   ⚠️  This will remove ALL session data")

print()

In [ ]:
# Troubleshooting Summary and Recommendations
print("📋 STEP 7: Troubleshooting Summary")
print("="*50)

print("🔍 Key Areas to Check for Secondary Tab Issues:")
print()

print("1. 🔒 Session Management:")
print("   - Check if _cleanup_inactive_sessions() is commented out")
print("   - Verify session isolation is working properly")
print("   - Ensure unique ports are allocated for each session")
print()

print("2. 🌐 Server Proxy Routing:")
print("   - Verify each session gets a unique /proxy/{port}/ path")
print("   - Check if JupyterLab can route to multiple proxy instances")
print("   - Test proxy path accessibility for each session")
print()

print("3. 🎛️  Frontend Integration:")
print("   - Check if JupyterLab frontend can handle multiple tabs")
print("   - Verify proper communication with backend API")
print("   - Test browser popup/tab creation mechanics")
print()

print("4. 🔧 Common Solutions:")
print("   - Enable session cleanup: Uncomment self._cleanup_inactive_sessions()")
print("   - Clear orphaned sessions manually if needed")
print("   - Restart JupyterLab to reset session state")
print("   - Check browser popup/tab permissions")
print()

print("5. 🐛 Quick Fixes to Try:")
print("   a) In firefox_handler.py, uncomment line:")
print("      self._cleanup_inactive_sessions()")
print()
print("   b) Clear all session data:")
print("      rm -rf ~/.firefox-launcher/sessions/*")
print()
print("   c) Restart JupyterLab completely")
print()
print("   d) Check browser settings - allow popups from JupyterLab")
print()

print("6. 🏥 Diagnostic Commands:")
print("   - Check active sessions: FirefoxLauncherHandler._active_sessions")
print("   - List processes: ps aux | grep -E '(firefox|xpra)'")
print("   - Check ports: netstat -tlnp | grep :8888")
print("   - View session dirs: ls -la ~/.firefox-launcher/sessions/")

print()
print("✅ Debug analysis complete!")
print("💡 Focus on session management and proxy routing for secondary tab issues.")

# Save debug info to file
debug_info = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "python_version": sys.version,
    "working_directory": os.getcwd(),
    "home_directory": str(Path.home()),
    "firefox_found": subprocess.run(['which', 'firefox'], capture_output=True).returncode == 0,
    "xpra_found": subprocess.run(['which', 'xpra'], capture_output=True).returncode == 0,
}

debug_file = Path("firefox_debug_info.json")
with open(debug_file, "w") as f:
    json.dump(debug_info, f, indent=2)

print(f"📄 Debug info saved to: {debug_file.absolute()}")